In [ ]:
'''''
Install a library called patchify to tile large images into small parts
Make two folders to hold images and masks
Make a list of tiff image file names in a given floder
Read tiff files one by one (each file differ by number of channels and where the color chanel is located on)
Use patchify to break the image to 4000 x 4000 tiles, do this by every 2000 pixels
Read the corresponding mask for the image given by json specification file
Generate mask tiles using patchify as per the same parameters
For each image and mask tile calculate the percentage of the mask covered in the whole image
Select all images where the mask is between 10-90% comapred the whole image
Resize each image and mask to 400 x 400
Rename those in a way that it can be identified uniquely using the row and column in which those are located in the original image
Save the resized image and mask in the corresponding folder
''''

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import cv2
import pickle
from datetime import datetime
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from keras.models import Model
import os
from keras.applications.vgg16 import VGG16
from keras.utils import normalize
from sklearn.model_selection import train_test_split
import random

In [3]:
pip install patchify

Note: you may need to restart the kernel to use updated packages.


In [2]:
from PIL import Image, ImageDraw
from patchify import patchify
import tifffile as tiff
import gc
import json
import csv
import math, re, os
import tensorflow as tf
from kaggle_datasets import KaggleDatasets
from tensorflow import keras
from functools import partial
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import keras
from keras.models import Sequential, Model,load_model
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from keras.preprocessing import image
from keras.initializers import glorot_uniform
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.4.1


In [5]:
#pip install imgaug
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug import augmenters as iaa
from imgaug import parameters as iap
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

In [6]:
def augment(image,mask):
    #image = m#ia.quokka(size=(128, 128), extract="square")
    n = mask
    segmap = (n>0).astype('uint8')
    segmap = SegmentationMapsOnImage(segmap, shape=image.shape)

    seq = iaa.Sequential([
        #iaa.Dropout([0.05, 0.2]),      # drop 5% or 20% of all pixels
        #iaa.Affine(rotate=(-45, 45)),  # rotate by -45 to 45 degrees (affects segmaps)
        #iaa.ElasticTransformation(alpha=50, sigma=5),  # apply water effect (affects segmaps)
        #iaa.GaussianBlur(sigma=iap.Uniform(0.9, 1.0)),
        iaa.MedianBlur(k=(3, 11))
    ], random_order=True)

    images_aug = []
    segmaps_aug = []
    #for _ in range(5):
    images_aug_i, segmaps_aug_i = seq(image=image, segmentation_maps=segmap)
    images_aug.append(images_aug_i)
    segmaps_aug.append(segmaps_aug_i)

    cells = []
    for image_aug, segmap_aug in zip(images_aug, segmaps_aug):
        cells.append(image)
        cells.append(n)
        cells.append(image_aug)
        cells.append(segmap_aug.draw(size=image_aug.shape[:2])[0])
    len(cells)

    #fig, axs = plt.subplots(1,4,figsize=(10,10))
    #axs[0].imshow(cells[0])
    #axs[1].imshow(cells[1])
    #axs[2].imshow(cells[2])
    #axs[3].imshow(cells[3])
    return cells[0],cells[1],cells[2],cells[3]

In [7]:
size_x = 4000
size_y = 4000

In [8]:
def file_list(folder):
    files_list = os.listdir('../input/hubmap-kidney-segmentation/'+folder)
    files = []
    for file in files_list:
        if ".tiff" in file:
            files.append(file.split(".")[0])
    return files

In [9]:
try:
    !rm -r images
    !rm -r masks
    print("Deleted")
except:
    pass
!mkdir images
!mkdir masks
print("created")

rm: cannot remove 'images': No such file or directory
rm: cannot remove 'masks': No such file or directory
Deleted
created


In [10]:
def read_tiff(file_id,folder):
    print("Reading file ",file_id,end="")
    large_image_stack = tiff.imread('../input/hubmap-kidney-segmentation/'+folder+'/'+file_id+'.tiff')#[0,0,:,:,:]
    print("Done")
    shape = large_image_stack.shape
    if(len(shape)>3):
        print("Image has more dims")
        large_image_stack = tiff.imread('../input/hubmap-kidney-segmentation/'+folder+'/'+file_id+'.tiff')[0,0,:,:,:]
    else:
        print("Image has 3 dims")
    shape = large_image_stack.shape
    if shape[0]==3:
        print("Image is channel first, converting")
        large_image_stack = np.einsum('ijk->jki',large_image_stack)
    else:
        print("Image is channel last, no need to convert")
    shape = large_image_stack.shape
    print(shape)
    return large_image_stack, shape

def read_mask(file_id):    
    json_filename = '../input/hubmap-kidney-segmentation/train/'+file_id+'.json'
    read_file = open(json_filename, "r") 
    data = json.load(read_file)

    polys = []
    for index in range(data.__len__()):
        geom = np.array(data[index]['geometry']['coordinates'])
        polys.append(geom)
    #shape = (38160, 39000)

    Image.MAX_IMAGE_PIXELS = None
    mask = Image.new('L', (shape[1], shape[0]), 0)  # (w, h)
    for i in range(len(polys)):
        poly = polys[i]
        ImageDraw.Draw(mask).polygon(tuple(map(tuple, poly[0])), outline=i + 1, fill=i + 1) 
    polys = []
    mask = np.array(mask)
    return mask

In [11]:
%%time
files = file_list('train')
c = 0
resize = size_x
patch_length = resize
step = 2000
resize = 256
items = 0
for file_id in files[:]:
    if items<10:
        items+=1
        continue
    large_image_stack, shape = read_tiff(file_id,"train")
    #large_image_stack = cv2.resize(large_image_stack,(shape[1]//3,shape[0]//3))
    print("New shape,",shape)
    patches_img = patchify(large_image_stack, (patch_length, patch_length, 3), step=step)
    large_image_stack = ""
    gc.collect()
    
    mask = read_mask(file_id)
    patches_mask = patchify(mask, (patch_length, patch_length), step=step)
    mask = ""
    gc.collect()
    
    count = 0
    sums = []
    for row in range(patches_mask.shape[0]):
        for col in range(patches_mask.shape[1]):
            s = np.sum(patches_mask[row,col,:,:])
            count+=1
            sums.append([s, row, col, np.round(s/(patch_length*patch_length*255),2)])
    sums.sort()
    #only select top 0-10% and 90%-100% images
    #sums = sums[0:int(len(sums)*.10)]+sums[int(len(sums)*.40):int(len(sums)*.60)]+(sums[int(len(sums)*.90):])
    c = 0
    for im in sums:
        c+=1
        #if c>20:
        row = im[1]
        col = im[2]

        image = patches_img[row,col,0,:,:,:]
        image = cv2.resize(image, (resize, resize))
        mask = patches_mask[row,col,:,:]
        mask = cv2.resize(mask, (resize, resize))
        mask = (mask>0).astype(int)

        org_img, org_msk, aug_img, aug_msk = augment(image,mask)

        filename_image = file_id+"_"+str("{0:0=3d}".format(row))+"_"+str("{0:0=3d}".format(col))+"_image.jpg"#file_id+"_"+str(str("{0:0=3d}".format(c)))+"image.jpg"
        cv2.imwrite("./images/"+filename_image, org_img)
        filename_mask = file_id+"_"+str("{0:0=3d}".format(row))+"_"+str("{0:0=3d}".format(col))+"_mask.jpg"#file_id+"_"+str(str("{0:0=3d}".format(c)))+"mask.jpg"
        cv2.imwrite("./masks/"+filename_mask, org_msk*255)

        num = random.randint(1,100)
        #print("Num is {}".format(num))
        if num>=50:
            filename_image = file_id+"_"+str("{0:0=3d}".format(row))+"_"+str("{0:0=3d}".format(col))+"_image_aug.jpg"#file_id+"_"+str(str("{0:0=3d}".format(c)))+"image.jpg"
            cv2.imwrite("./images/"+filename_image, aug_img)
            filename_mask = file_id+"_"+str("{0:0=3d}".format(row))+"_"+str("{0:0=3d}".format(col))+"_mask_aug.jpg"#file_id+"_"+str(str("{0:0=3d}".format(c)))+"mask.jpg"
            cv2.imwrite("./masks/"+filename_mask, org_msk*255)
        print(".",end="")
            
            #if im[3]>=.30: break
            #print(row, col, filename_image, filename_mask, "saved")
    print()
    print(items," ","Done",c," images")
    print()
    items+=1
    gc.collect()
    
print("DONE=================")

Reading file  8242609faDone
Image has 3 dims
Image is channel last, no need to convert
(31299, 44066, 3)
New shape, (31299, 44066, 3)
......................................................................................................................................................................................................................................................................................................
10   Done 294  images

Reading file  1e2425f28Done
Image has 3 dims
Image is channel first, converting
(26780, 32220, 3)
New shape, (26780, 32220, 3)
....................................................................................................................................................................................
11   Done 180  images

Reading file  b2dc8411cDone
Image has 3 dims
Image is channel last, no need to convert
(14844, 31262, 3)
New shape, (14844, 31262, 3)
...................................................................................